# Country Club Analysis with SQL 

Our new country club has been open for 3 months! It's time to analyze the data from our first quarter and evaulate the state of the business. 

In [1]:
# Import packages

import pandas as pd
import sqlite3

In [339]:
# Connect to database

conn = sqlite3.connect("country_club_database.db")
cur = conn.cursor() 

In [340]:
# View all tables in database

tables = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

print(f'The following tables are in this database:')
for i in tables:
    print(i[0])


The following tables are in this database:
Bookings
Facilities
Members


Let's examine our  3 tables:

In [377]:
# The facilities table

query = ''' SELECT *
            FROM facilities;
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.000000,25.000000,10000,200
1,Tennis Court 2,5.000000,25.000000,8000,200
2,Badminton Court,0.000000,15.500000,4000,50
3,Table Tennis,0.000000,5.000000,320,10
4,Massage Room 1,9.900000,80.000000,4000,3000
5,Massage Room 2,9.900000,80.000000,4000,3000
6,Squash Court,3.500000,17.500000,5000,80
7,Snooker Table,0.000000,5.000000,450,15
8,Pool Table,0.000000,5.000000,400,15


There are a total of 9 facilities at the country club.

In [400]:
# The bookings table 

query = '''SELECT *
           FROM bookings;
        
        '''

df = pd.read_sql_query(query, conn)
df = df.head(20)  # returns summary of dataset 
df.style.hide_index()



bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2
3,7,1,2012-07-03 19:00:00,2
4,8,1,2012-07-03 10:00:00,1
5,8,1,2012-07-03 15:00:00,1
6,0,2,2012-07-04 09:00:00,3
7,0,2,2012-07-04 15:00:00,3
8,4,3,2012-07-04 13:30:00,2
9,4,0,2012-07-04 15:00:00,2


Our booking data spans from July 3, 2012 through September 30, 2012.

In [334]:
# The members table 

query = '''SELECT *
           FROM members;
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,,2012-07-15 08:52:55
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05


In [342]:
# Show member count 

query = '''SELECT COUNT(DISTINCT memid)
           FROM members
           WHERE memid != 0;
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

COUNT(DISTINCT memid)
30


We have 30 members and allow for guest visits.

Now let's write some queries to produce datasets of interest. 



In [425]:
# Total revenue and overall profitability for the July - September period 

query = ''' WITH rev_prof AS 
            (SELECT f.name AS facility,
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) AS revenue_per_fac,
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) - (monthlymaintenance * 3) - (initialoutlay) AS profit
            FROM facilities AS f
            INNER JOIN bookings ON f.facid = bookings.facid
            GROUP BY facility 
            ORDER BY profit DESC),

            expenses AS 
            (SELECT SUM(initialoutlay) AS total_outlay, SUM(monthlymaintenance * 3) AS total_maintenance
            FROM facilities)
            
            
            SELECT SUM(revenue_per_fac) AS total_revenue, 
                   total_outlay, total_maintenance, 
                   SUM(profit) AS total_profit
            FROM rev_prof,
                 expenses;
       '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

total_revenue,total_outlay,total_maintenance,total_profit
109040.700000,36170,19710,53160.700000


Overall, we've made a profit of $53,160! Not bad for the first 3 months.

In [418]:
# Examine the total revenue and profit for each facility for the period in question (July - Sep)
# Revenue minus monthly maintenance(times 3) & initial investment (initial outlay) = profit

query = ''' SELECT f.name AS facility,
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) AS revenue,
            f.initialoutlay AS initial_investment,   
            f.monthlymaintenance * 3 AS maintenance_for_period,     
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) - (monthlymaintenance * 3) - (initialoutlay) AS profit
            FROM Facilities AS f
            INNER JOIN bookings ON f.facid = bookings.facid
            GROUP BY facility 
            ORDER BY profit DESC;
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

facility,revenue,initial_investment,maintenance_for_period,profit
Massage Room 1,50351.600000,4000,9000,37351.600000
Squash Court,13468.000000,5000,240,8228.000000
Tennis Court 2,14310.000000,8000,600,5710.000000
Tennis Court 1,13860.000000,10000,600,3260.000000
Massage Room 2,14454.600000,4000,9000,1454.600000
Table Tennis,180.000000,320,30,-170.000000
Pool Table,270.000000,400,45,-175.000000
Snooker Table,240.000000,450,45,-255.000000
Badminton Court,1906.500000,4000,150,-2243.500000


The most profitable facilities are Massage Room 1, Squash Court and Tennis Court 2 for the time period noted.  In contrast, the facilities with the greatest loss for the time period are the Badminton Court, Snooker Table and Pool Table (in order of greatest to least). Contributors to loss in each of these facilities can be attributed to pricing for members and guests.  

Our steepest loss (by far) is the badminton court. This is because the initial investment was $ 4000 (the same as the massages rooms). However, it's free for members and only $15 for guests. Whereas the massage room is $10 for members and $80 for guests. The monthly maintenance for the badminton court is only $50 vs $3000 for the massage rooms, so we expect it to become profitable over time due to low mainatenance cost. However we may want to look into increasing the price. 

Let's take a look at the average amount of bookings per member:

In [351]:
# The count of bookings divided by the count of unique member id's

query = ''' SELECT (COUNT(b.bookid) / COUNT(DISTINCT memid)) AS bookings_per_member
            FROM bookings AS b;
            
        '''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

bookings_per_member
134


Let's calculate the revenue per member:

In [395]:
# Total revenue divided by total members 

query = ''' WITH revenue AS 
            (SELECT f.name AS facility,
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) AS revenue_per_fac
            FROM Facilities AS f
            INNER JOIN bookings ON f.facid = bookings.facid
            GROUP BY facility),

            member_count AS
            (SELECT COUNT(DISTINCT m.memid) AS mem
            FROM members AS m
            WHERE m.memid != 0 )

            SELECT (SUM(revenue_per_fac) /  mem)  AS revenue_per_member
            FROM revenue,
            member_count;
            
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

revenue_per_member
3634.690000


Let's calculate the revenue per booking:

In [404]:
# Total revenue divided by total bookings 

query = ''' WITH revenue AS 
            (SELECT f.name AS facility,
            SUM(CASE WHEN bookings.memid != 0 THEN f.membercost * bookings.slots
    	            ELSE f.guestcost * bookings.slots END) AS revenue_per_fac
            FROM Facilities AS f
            INNER JOIN bookings ON f.facid = bookings.facid
            GROUP BY facility),

            booking_count AS
            (SELECT COUNT(DISTINCT bookid) AS count_
            FROM bookings)

            SELECT (SUM(revenue_per_fac) / count_) AS revenue_per_booking
            FROM revenue,
            booking_count;
            
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

revenue_per_booking
26.970245


Excellent! 

We are making $3,634 per member and $27 per booking.

Now that we've examined our revenue/profit, let's make some more inquiries:

In [93]:
# We are considering making some design changes to the the tennis courts. 
# Let's produce a list of all members who have used a tennis court to send out a phone survey. 

query = '''SELECT DISTINCT m.firstname || ' ' || m.surname AS name,
                  m.telephone AS contact,
		          Facilities.name AS facility
            FROM Members AS m 
            JOIN Bookings ON m.memid = Bookings.memid
            JOIN Facilities ON Facilities.facid = Bookings.facid
            WHERE Facilities.name LIKE 'Tennis%' AND m.firstname NOT LIKE 'Guest%'
            GROUP BY m.firstname
            ORDER BY name;
        '''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

name,contact,facility
Anne Baker,844-076-5141,Tennis Court 1
Burton Tracy,(822) 354-9973,Tennis Court 2
Charles Owen,(855) 542-5251,Tennis Court 1
Darren Smith,555-555-5555,Tennis Court 2
David Jones,(844) 536-8036,Tennis Court 2
Douglas Jones,844 536-8036,Tennis Court 1
Erica Crumpet,(811) 732-4816,Tennis Court 1
Florence Bader,(833) 499-3527,Tennis Court 2
Gerald Butters,(844) 078-4130,Tennis Court 1
Henrietta Rumney,(822) 989-8876,Tennis Court 2


In [142]:
# We would like to send out appreciation cards to members who have recommended others who in turn became new members!
# Let's produce a report of members and those who recommended them in alphabetical order

query = '''SELECT m.firstname || ' ' || m.surname AS 'new member',
		  m1.firstname || ' ' || m1.surname AS 'recommended by',
                  m1.address 
           FROM Members as m
           INNER JOIN Members as m1 
           ON m.recommendedby = m1.memid
           WHERE m1.memid != 0
           ORDER BY m.surname, m.firstname;
        '''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

new member,recommended by,address
Florence Bader,Ponder Stibbons,"5 Dragons Way, Winchester"
Anne Baker,Ponder Stibbons,"5 Dragons Way, Winchester"
Timothy Baker,Jemima Farrell,"103 Firth Avenue, North Reading"
Tim Boothe,Tim Rownam,"23 Highway Way, Boston"
Gerald Butters,Darren Smith,"8 Bloomsbury Close, Boston"
Joan Coplin,Timothy Baker,"329 James Street, Reading"
Erica Crumpet,Tracy Smith,"8 Bloomsbury Close, New York"
Nancy Dare,Janice Joplette,"20 Crossing Road, New York"
Matthew Genting,Gerald Butters,"1065 Huntingdon Avenue, Boston"
John Hunt,Millicent Purview,"641 Drudgery Close, Burnington, Boston"


In [237]:
# We would like to find the most popular facilities amongst our members
# Let's produce a list of the facilities with their total usage (booking count) over the 3 month period (guests not included)

query = ''' SELECT f.name AS 'Name', 
                   b.total AS Booking_count
            FROM Facilities AS f
            LEFT JOIN (SELECT facid, COUNT(bookid) AS total
            FROM Bookings AS b
            WHERE memid != 0
            GROUP BY facid) AS b 
            ON f.facid = b.facid
            ORDER BY Booking_count DESC;
        '''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

Name,Booking_count
Pool Table,783
Massage Room 1,421
Snooker Table,421
Table Tennis,385
Badminton Court,344
Tennis Court 1,308
Tennis Court 2,276
Squash Court,195
Massage Room 2,27


In [408]:
# Now let's find the most popular facilities amongst our members per month!

query = ''' SELECT CASE WHEN strftime('%m', b.starttime) = '07' THEN 'July'
                        WHEN strftime('%m', b.starttime) = '08' THEN 'August'
                        ELSE 'September' END AS Month, 
                    f.name AS Facility, 
                    COUNT(b.slots) AS Monthly_Usage
            FROM Bookings AS b
            LEFT JOIN Facilities AS f 
            ON f.facid = b.facid
            WHERE b.memid != 0      
            GROUP BY f.name, month 
            ORDER BY strftime('%m', b.starttime), Monthly_Usage DESC;
        '''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

Month,Facility,Monthly_Usage
July,Pool Table,103
July,Massage Room 1,77
July,Snooker Table,68
July,Tennis Court 1,65
July,Badminton Court,51
July,Table Tennis,48
July,Tennis Court 2,41
July,Squash Court,23
July,Massage Room 2,4
August,Pool Table,272


The Pool Tabel is consistently the most popular facility. This might be due to its low cost, popularity as a game, and the ability for people to socialize while playling. 

In [409]:
# Let's view all bookings for September 14 where the booking cost was over $30

query = ''' SELECT b.bookid,
                   firstname || ' ' || surname AS Name,
                   f.name AS facility,
            CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                 ELSE f.membercost * b.slots END AS cost_of_booking
            FROM bookings AS b 
            JOIN facilities AS f ON b.facid = f.facid
            JOIN members AS m ON b.memid = m.memid
            WHERE b.starttime LIKE '2012-09-14%' AND cost_of_booking > 30
            ORDER BY cost_of_booking DESC;
        '''


df = pd.read_sql_query(query, conn)
df.style.hide_index()


bookid,Name,facility,cost_of_booking
2946,GUEST GUEST,Massage Room 2,320.000000
2937,GUEST GUEST,Massage Room 1,160.000000
2940,GUEST GUEST,Massage Room 1,160.000000
2942,GUEST GUEST,Massage Room 1,160.000000
2926,GUEST GUEST,Tennis Court 2,150.000000
2920,GUEST GUEST,Tennis Court 1,75.000000
2922,GUEST GUEST,Tennis Court 1,75.000000
2925,GUEST GUEST,Tennis Court 2,75.000000
2948,GUEST GUEST,Squash Court,70.000000
2941,Jemima Farrell,Massage Room 1,39.600000


On any given day, our guests pay more for services. This is simply because the pricing is significantly higher for non-members. 